In [ ]:
pip install flask-ngrok

In [ ]:
! wget -q -c -nc https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip -qq -n ngrok-stable-linux-amd64.zip

In [ ]:
print("Get your authtoken from https://dashboard.ngrok.com/auth")
import getpass
authtoken = getpass.getpass()

Get your authtoken from https://dashboard.ngrok.com/auth
··········


In [ ]:
get_ipython().system_raw('./ngrok authtoken $authtoken && ./ngrok tcp 22 &')

In [ ]:
cd /content/drive/MyDrive/Colab Notebooks/Flask-deploy-model

/content/drive/MyDrive/Colab Notebooks/Flask-deploy-model


In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask, request, render_template, redirect
import pickle
import joblib
#import numpy as np
import pandas as pd

In [ ]:
#Create Flask App
app = Flask(__name__)

#Run Flask App With Ngrok
run_with_ngrok(app)

model = joblib.load(open('finalized_model.pkl','rb'))

@app.route('/')
def index():
  return render_template('upload.html')

@app.route("/upload", methods = ["GET", "POST"])
def upload():
  
  if request.method == "POST":
    if request.files:
      output = ""
      csv_file = request.files.get("file")
      df = pd.read_csv(csv_file)
      df.drop(labels = ['Unnamed: 0'], inplace = True, axis = 1)
      tdf = df.values
      result = model.predict(tdf)
      temp = [str(i) for i in result]
      for i in range (len(temp)):
        if temp[i] == '1':
          temp[i] = 'Malignant'
        else:
          temp[i] = 'Benign'
      output = temp[0]

      return render_template('upload.html', output='Your Diagnosis Result are {}'.format(output), 
      tables=[df.to_html(classes='data')])

  return render_template("upload.html")

if __name__ == "__main__":
  app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://cce4-34-73-61-87.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [19/May/2022 17:22:09] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/May/2022 17:22:09] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [19/May/2022 17:22:14] "POST /upload HTTP/1.1" 200 -
